[![](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/redun-lamin-fasta/blob/main/docs/redun.ipynb)
[![](https://img.shields.io/badge/Source%20%26%20report%20on%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/transform/taasWKawCiNA6zf0)

# Redun

Here, we'll see how to track redun workflow runs with LaminDB.

```{note}

This use case is based on [github.com/ricomnl/bioinformatics-pipeline-tutorial](https://github.com/ricomnl/bioinformatics-pipeline-tutorial/tree/redun).

```

In [ ]:
!lamin init --storage ./test-redun-lamin --schema bionty

## Amend the workflow

In [ ]:
import lamindb as ln
import json

Let's amend a redun `workflow.py` to register input & output artifacts in LaminDB:

- To track the workflow run in LaminDB, add ([see on GitHub](https://github.com/laminlabs/redun-lamin-fasta/blob/main/docs/workflow.py#L51)):

    ```python
    ln.track(params=params)
    ```

- To register the output file via LaminDB, add ([see on GitHub](https://github.com/laminlabs/redun-lamin-fasta/blob/main/redun_lamin_fasta/lib.py#L315)):

    ```python
    ln.Artifact(output_path, description="results").save()
    ```

## Run redun

Let's see what the input files are:

In [ ]:
!ls ./fasta

And call the workflow:

In [ ]:
!redun run workflow.py main --input-dir ./fasta --tag run=test-run  1> redun_stdout.txt 2>redun_stderr.txt

Inspect the output:

In [ ]:
!cat redun_stdout.txt

And the error log:

In [ ]:
!tail -1 redun_stderr.txt

View data lineage:

In [ ]:
artifact = ln.Artifact.filter(description="results", suffix=".tgz").one()
artifact.view_lineage()

## Track the redun execution id

If we want to be able to query LaminDB for redun execution ID, this here is a way to get it:

In [ ]:
# export the run information from redun
!redun log --exec --exec-tag run=test-run --format json --no-pager > redun_exec.json
# load the redun execution id from the JSON and store it in the LaminDB run record
with open("redun_exec.json") as file:
    redun_exec = json.loads(file.readline())
artifact.run.reference = redun_exec["id"]
artifact.run.reference_type = "redun_id"
artifact.run.save()

## Track the redun run report

Attach a run report:

In [ ]:
report = ln.Artifact(
    "redun_stderr.txt",
    description=f"Redun run report of {redun_exec['id']}",
    run=False,
    visibility=0,
).save()
artifact.run.report = report
artifact.run.save()

## View transforms and runs in LaminHub

[![hub](https://img.shields.io/badge/View%20in%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/transform/taasWKawCiNA6zf0/pe5H0WQtlTDmqKoEyO6k)

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/xzIfn0p1wAqXH1TZF5J5.png" width="900px">

## View the database content

In [ ]:
ln.view()

Delete the test instance:

In [ ]:
!rm -rf test-redun-lamin
!lamin delete --force test-redun-lamin